In [ ]:
# # ================================
# # STEP 1: Install Dependencies
# # ================================

%%capture
# This command forces an upgrade of the core conflicting library (NumPy).
!pip install --upgrade -q pip numpy

# Now, install the rest of the application's dependencies
!pip install -q fastapi uvicorn python-multipart PyMuPDF "pinecone-client>=3.2.2" "langchain>=0.1.16" "langchain-community>=0.0.32" langchain-huggingface langchain-pinecone "transformers>=4.40.1" torch torchvision torchaudio --upgrade "accelerate>=0.29.3" "pyngrok>=7.1.6" "nest_asyncio>=1.6.0"

In [ ]:
!pip install --upgrade llama-index langchain langchain-huggingface transformers sentence-transformers
!pip uninstall -y llama-index
!pip install --upgrade langchain transformers sentence-transformers pinecone-client
!pip install "llama-index==0.5.6"

In [ ]:

%%capture
## Install a specific, known-good set of libraries to ensure full compatibility.
!pip install -q fastapi uvicorn python-multipart PyMuPDF "pinecone-client==3.2.2" "langchain==0.2.5" "langchain-community==0.2.5" "langchain-huggingface==0.0.3" "langchain-pinecone==0.1.1" "transformers==4.40.1" "torch==2.3.1" "torchvision==0.18.1" "torchaudio==2.3.1" --index-url https://download.pytorch.org/whl/cu121 "accelerate==0.29.3" "pyngrok>=7.1.6" "nest_asyncio>=1.6.0"

In [ ]:
!pip install --upgrade transformers
!pip install -q bitsandbytes==0.43.1
!pip install -q triton
!pip install --upgrade bitsandbytes
# Run this cell once to install the Tesseract OCR engine
!sudo apt install tesseract-ocr
!pip install pytesseract



In [ ]:
!ngrok config add-authtoken -KEY-


In [ ]:
from huggingface_hub import notebook_login

# This will display a login widget to authenticate your notebook.
notebook_login()

In [ ]:
# ================================
# IMPORTS & SETUP
# ================================
import os
import logging
import nest_asyncio
import fitz  # PyMuPDF
from pyngrok import ngrok
import uuid
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import pytesseract
from PIL import Image
import io

# LangChain Imports
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

from dotenv import load_dotenv

# Apply nest_asyncio to allow asyncio to run in a notebook environment
nest_asyncio.apply()

# Load environment variables from a .env file if it exists
load_dotenv()

# ================================
# CONFIGURATION
# ================================
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Securely get API keys from environment variables
HF_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not HF_KEY or not PINECONE_API_KEY:
    logging.warning("⚠️ Hugging Face or Pinecone API keys are not set. Please set them as environment variables.")

PINECONE_INDEX_NAME = "knowledge-assistant"
app = FastAPI()

# ================================
# HTML FRONTEND (With Session Management)
# ================================
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Spe | Knowledge Assistant</title>
    <style>
        :root {
            --bg-color: #0c0a1d;
            --container-bg: rgba(26, 16, 60, 0.85);
            --primary-accent: #9d55f5;
            --secondary-accent: #f555b5;
            --text-color: #e0e0e0;
            --text-color-dark: #1a103c;
            --border-color: #4d3e8c;
            --user-msg-bg: #4a3a8a;
            --assistant-msg-bg: #3a2d7a;
            --font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif;
            --border-radius: 16px;
            --box-shadow: 0 10px 30px rgba(0,0,0,0.4);
        }
        body {
            font-family: var(--font-family); margin: 0; color: var(--text-color);
            display: flex; justify-content: center; align-items: center;
            height: 100vh; padding: 16px; box-sizing: border-box;
            background: var(--bg-color);
            overflow: hidden; /* Prevents scrollbars from the animated background */
        }
        .stars {
            position: fixed; top: 0; left: 0; width: 100%; height: 100%;
            background-image:
                radial-gradient(2px 2px at 20px 30px, #eee, rgba(0,0,0,0)),
                radial-gradient(2px 2px at 40px 70px, #fff, rgba(0,0,0,0)),
                radial-gradient(2px 2px at 50px 160px, #ddd, rgba(0,0,0,0)),
                radial-gradient(2px 2px at 90px 40px, #fff, rgba(0,0,0,0)),
                radial-gradient(2px 2px at 130px 80px, #fff, rgba(0,0,0,0)),
                radial-gradient(2px 2px at 160px 120px, #ddd, rgba(0,0,0,0));
            background-repeat: repeat;
            background-size: 200px 200px;
            animation: zoom 15s infinite;
            opacity: 0;
            animation-delay: 1s;
            animation-timing-function: linear;
        }
        @keyframes zoom {
            0% { transform: scale(1); opacity: 0; }
            50% { opacity: 0.7; }
            100% { transform: scale(2); opacity: 0; }
        }
        .chat-container {
            width: 100%; max-width: 800px; height: 95vh; display: flex;
            flex-direction: column; background: var(--container-bg); border-radius: var(--border-radius);
            box-shadow: var(--box-shadow); overflow: hidden; border: 1px solid var(--border-color);
            backdrop-filter: blur(10px); z-index: 1;
        }
        header {
            background: linear-gradient(90deg, var(--primary-accent), var(--secondary-accent));
            padding: 16px; text-align: center; font-size: 1.25rem; font-weight: 600;
            display: flex; align-items: center; justify-content: center; gap: 12px;
            color: var(--text-color-dark); text-shadow: 1px 1px 2px rgba(0,0,0,0.2);
            flex-shrink: 0;
        }
        .chat-box { flex-grow: 1; padding: 24px; overflow-y: auto; display: flex; flex-direction: column; gap: 16px; }
        .message { display: flex; align-items: flex-start; gap: 12px; max-width: 85%; animation: fadeIn 0.3s ease-in-out; }
        @keyframes fadeIn { from { opacity: 0; transform: translateY(10px); } to { opacity: 1; transform: translateY(0); } }
        .message .avatar { width: 40px; height: 40px; border-radius: 50%; display: flex; justify-content: center; align-items: center; font-weight: bold; flex-shrink: 0; }
        .message .content { padding: 12px 16px; border-radius: var(--border-radius); white-space: pre-wrap; word-wrap: break-word; }
        .user-message { align-self: flex-end; flex-direction: row-reverse; }
        .user-message .avatar { background-color: var(--secondary-accent); color: var(--text-color-dark); }
        .user-message .content { background-color: var(--user-msg-bg); border-bottom-right-radius: 4px; }
        .assistant-message { align-self: flex-start; }
        .assistant-message .avatar { background-color: var(--primary-accent); color: var(--text-color-dark); }
        .assistant-message .content { background-color: var(--assistant-msg-bg); border-bottom-left-radius: 4px; }
        .sources { margin-top: 12px; font-size: 0.85em; }
        .sources-title { font-weight: bold; margin-bottom: 4px; color: var(--secondary-accent); }
        .source-tag { display: inline-block; background-color: var(--user-msg-bg); padding: 3px 10px; border-radius: 12px; margin: 0 5px 5px 0; font-size: 0.8rem; }
        .chat-input { display: flex; padding: 16px; border-top: 1px solid var(--border-color); gap: 10px; align-items: center; flex-shrink: 0; }
        .chat-input input {
            flex-grow: 1; padding: 12px 18px; border: 1px solid var(--border-color); border-radius: 25px;
            font-size: 1rem; background-color: var(--bg-color); color: var(--text-color); transition: all 0.2s ease;
        }
        .chat-input input:focus { outline: none; border-color: var(--primary-accent); box-shadow: 0 0 0 3px rgba(157, 85, 245, 0.4); }
        .chat-input button {
            width: 45px; height: 45px; border: none; border-radius: 50%;
            cursor: pointer; display: flex; justify-content: center; align-items: center; transition: all 0.2s ease; flex-shrink: 0;
        }
        .chat-input button.send-button { background-color: var(--primary-accent); color: var(--text-color-dark); }
        .chat-input button.send-button:hover { background-color: #b37af8; transform: scale(1.1); }
        .chat-input button:disabled { background-color: #555; cursor: not-allowed; transform: none; }
        .chat-input button.stop-button { background-color: var(--secondary-accent); color: var(--text-color-dark); }
        .chat-input button.stop-button:hover { background-color: #f87ac5; transform: scale(1.1); }
        .chat-input button.add-button { background-color: var(--user-msg-bg); color: var(--text-color); }
        .chat-input button.add-button:hover { background-color: #6a5a9a; transform: scale(1.1); }
        .upload-overlay {
            position: absolute; inset: 0; background: rgba(12, 10, 29, 0.95); backdrop-filter: blur(5px); z-index: 10;
            display: flex; justify-content: center; align-items: center; flex-direction: column; padding: 24px;
        }
        .upload-container { width: 100%; max-width: 500px; text-align: center; background: var(--container-bg); padding: 30px; border-radius: var(--border-radius); border: 1px solid var(--border-color); }
        #drop-zone { border: 2px dashed var(--border-color); border-radius: var(--border-radius); padding: 40px; color: var(--accent-color); transition: border-color 0.3s, background-color 0.3s; box-sizing: border-box; }
        #drop-zone.dragover { border-color: var(--primary-accent); background-color: rgba(157, 85, 245, 0.1); }
        #file-list { margin-top: 16px; max-height: 150px; overflow-y: auto; text-align: left; font-size: 0.9em; }
        .primary-button { background: linear-gradient(90deg, var(--primary-accent), var(--secondary-accent)); color: var(--text-color-dark); padding: 12px 24px; font-size: 1.1rem; border: none; border-radius: 25px; cursor: pointer; font-weight: 600; transition: all 0.2s ease; }
        .primary-button:hover { transform: scale(1.05); box-shadow: 0 5px 15px rgba(0,0,0,0.3); }
        .primary-button:disabled { background: #555; color: #888; cursor: not-allowed; transform: none; box-shadow: none; }
        .heartbeat-loader { width: 50px; height: 50px; position: relative; }
        .heartbeat-loader svg { width: 100%; height: 100%; position: absolute; top: 0; left: 0; }
        .heartbeat-loader path {
            stroke: var(--secondary-accent); stroke-width: 3; fill: transparent;
            stroke-dasharray: 1000; stroke-dashoffset: 1000;
            animation: draw 2s linear infinite, pulse 2s ease-in-out infinite;
        }
        @keyframes draw { to { stroke-dashoffset: 0; } }
        @keyframes pulse {
            0%, 100% { transform: scale(0.95); }
            50% { transform: scale(1.1); }
        }
        @media (max-width: 768px) {
            body { padding: 0; }
            .chat-container {
                height: 100vh; max-height: none;
                border-radius: 0; border: none;
            }
            .chat-box { padding: 16px; }
            .chat-input { padding: 12px; }
        }
    </style>
</head>
<body>
    <div class="stars"></div>
    <div class="chat-container">
        <header>
            <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M12 22a10 10 0 1 0 0-20 10 10 0 0 0 0 20Z"/><path d="M12 16a4 4 0 1 0 0-8 4 4 0 0 0 0 8Z"/><path d="M12 12v1"/><path d="M12 8V7"/><path d="m16 12 1-.01"/><path d="m8 12-1-.01"/></svg>
            <span>Spe - Knowledge Assistant</span>
        </header>
        <div class="chat-box" id="chat-box"></div>
        <div class="chat-input">
            <button id="add-button" onclick="startOver()" class="add-button" style="display: none;" title="Upload new documents">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><line x1="12" y1="5" x2="12" y2="19"></line><line x1="5" y1="12" x2="19" y2="12"></line></svg>
            </button>
            <input type="text" id="query-input" placeholder="Upload documents to begin..." disabled>
            <button id="send-button" onclick="askQuestion()" class="send-button" disabled>
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><line x1="22" y1="2" x2="11" y2="13"></line><polygon points="22 2 15 22 11 13 2 9 22 2"></polygon></svg>
            </button>
            <button id="stop-button" onclick="stopGeneration()" class="stop-button" style="display: none;" title="Stop generation">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="currentColor" stroke="none"><path d="M6 6h12v12H6z"></path></svg>
            </button>
        </div>
        <div class="upload-overlay" id="upload-overlay">
            <div class="upload-container">
                <h2>Upload Knowledge Base</h2>
                <p>Begin by uploading PDF or image files.</p>
                <div id="drop-zone">
                    <p><strong>Drag & Drop files here</strong> or</p>
                    <input type="file" id="file-upload" multiple accept=".pdf,.png,.jpg,.jpeg" hidden>
                    <button onclick="document.getElementById('file-upload').click()" class="primary-button" style="background: var(--assistant-msg-bg); color: white;">Select Files</button>
                    <div id="file-list"></div>
                </div>
                <button id="upload-button" onclick="uploadFiles()" class="primary-button" style="margin-top: 20px;" disabled>Upload & Process</button>
                <div id="upload-status" style="margin-top: 16px;"></div>
            </div>
        </div>
    </div>
    <script>
        const chatBox = document.getElementById('chat-box');
        const queryInput = document.getElementById('query-input');
        const sendButton = document.getElementById('send-button');
        const stopButton = document.getElementById('stop-button');
        const uploadOverlay = document.getElementById('upload-overlay');
        const dropZone = document.getElementById('drop-zone');
        const fileInput = document.getElementById('file-upload');
        const fileList = document.getElementById('file-list');
        const uploadButton = document.getElementById('upload-button');
        const uploadStatus = document.getElementById('upload-status');
        const addButton = document.getElementById('add-button');

        let currentTypewriterInterval = null;

        // ✅ NEW: Generate a UUID for the session
        const uuidv4 = () => {
            return ([1e7]+-1e3+-4e3+-8e3+-1e11).replace(/[018]/g, c =>
                (c ^ crypto.getRandomValues(new Uint8Array(1))[0] & 15 >> c / 4).toString(16)
            );
        };
        let currentSessionId = uuidv4();

        dropZone.addEventListener('dragover', (e) => { e.preventDefault(); dropZone.classList.add('dragover'); });
        dropZone.addEventListener('dragleave', () => dropZone.classList.remove('dragover'));
        dropZone.addEventListener('drop', (e) => { e.preventDefault(); dropZone.classList.remove('dragover'); handleFiles(e.dataTransfer.files); });
        fileInput.addEventListener('change', () => handleFiles(fileInput.files));

        function handleFiles(files) {
            fileList.innerHTML = '';
            Array.from(files).forEach(file => fileList.innerHTML += `<p style="margin: 4px 0;">${file.name}</p>`);
            uploadButton.disabled = files.length === 0;
        }

        function startOver() {
            // ✅ NEW: Generate a new session ID to start fresh
            currentSessionId = uuidv4();

            uploadOverlay.style.display = 'flex';
            chatBox.innerHTML = '';
            queryInput.disabled = true;
            queryInput.placeholder = 'Upload new documents to begin...';
            sendButton.disabled = true;
            addButton.style.display = 'none';
            fileInput.value = '';
            fileList.innerHTML = '';
            uploadButton.disabled = true;
            uploadStatus.innerHTML = '';
        }

        async function uploadFiles() {
            const files = Array.from(fileInput.files);
            if (files.length === 0) return;

            const formData = new FormData();
            files.forEach(file => formData.append('files', file));
            // ✅ NEW: Send the session ID with the upload request
            formData.append('session_id', currentSessionId);

            uploadStatus.innerHTML = `
                <div class="heartbeat-loader">
                    <svg viewBox="0 0 100 100">
                        <path d="M 90 40 C 90 20 70 20 70 40 C 70 50 90 65 90 80 C 90 95 70 95 70 80 C 50 65 50 50 50 40 C 50 20 30 20 30 40 C 30 50 10 65 10 80 C 10 95 30 95 30 80 C 50 65 50 50 50 40" />
                    </svg>
                </div>
                <p>Processing files...</p>`;
            uploadButton.disabled = true;

            try {
                const response = await fetch('/upload/', { method: 'POST', body: formData });
                const result = await response.json();
                if (!response.ok) throw new Error(result.detail || 'Upload failed.');

                uploadOverlay.style.display = 'none';
                queryInput.disabled = false;
                sendButton.disabled = false;
                addButton.style.display = 'flex';
                queryInput.placeholder = 'Ask a question about your documents...';
                queryInput.focus();
                addMessage('assistant', 'Hello! Your documents are ready. How can I help you?');
            } catch (error) {
                const errorMsg = `❌ Error: ${error.message}`;
                uploadStatus.innerHTML = `<span style="color: #ff6b6b;">${errorMsg}</span>`;
                uploadButton.disabled = false;
            }
        }

        async function askQuestion() {
            const query = queryInput.value.trim();
            if (!query) return;

            addMessage('user', query);
            queryInput.value = '';
            queryInput.focus();

            sendButton.style.display = 'none';
            stopButton.style.display = 'flex';

            const aiMessageContent = addMessage('assistant', '');

            try {
                // ✅ NEW: Send the session ID with the query
                const payload = {
                    query: query,
                    session_id: currentSessionId
                };
                const response = await fetch('/query/', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(payload),
                });
                const result = await response.json();
                if (!response.ok) throw new Error(result.detail || 'Query failed.');

                typewriterEffect(aiMessageContent, result.answer, result.sources);
            } catch (error) {
                typewriterEffect(aiMessageContent, `❌ An error occurred: ${error.message}`);
            }
        }

        function stopGeneration() {
            if (currentTypewriterInterval) {
                clearInterval(currentTypewriterInterval);
                currentTypewriterInterval = null;
                const lastMessageContent = document.querySelector('.assistant-message:last-child .content');
                if (lastMessageContent) {
                    lastMessageContent.innerHTML += '... [Stopped]';
                }
            }
            resetButtons();
        }

        function resetButtons() {
            stopButton.style.display = 'none';
            sendButton.style.display = 'flex';
        }

        function typewriterEffect(element, text, sources = []) {
            const words = text.split(' ');
            let i = 0;
            element.textContent = '';

            currentTypewriterInterval = setInterval(() => {
                if (i < words.length) {
                    element.textContent += words[i] + ' ';
                    chatBox.scrollTop = chatBox.scrollHeight;
                    i++;
                } else {
                    clearInterval(currentTypewriterInterval);
                    currentTypewriterInterval = null;
                    if (sources && sources.length > 0) {
                        const sourcesDiv = document.createElement('div');
                        sourcesDiv.classList.add('sources');
                        sourcesDiv.innerHTML = `<div class="sources-title">Sources:</div>`;
                        sources.forEach(source => {
                            sourcesDiv.innerHTML += `<span class="source-tag">${source}</span>`;
                        });
                        element.appendChild(sourcesDiv);
                        chatBox.scrollTop = chatBox.scrollHeight;
                    }
                    resetButtons();
                }
            }, 50);
        }

        function addMessage(sender, text) {
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', `${sender}-message`);

            const avatar = document.createElement('div');
            avatar.classList.add('avatar');
            avatar.textContent = sender === 'user' ? 'You' : 'S';

            const content = document.createElement('div');
            content.classList.add('content');
            if (text) {
                content.textContent = text;
            }

            messageDiv.appendChild(avatar);
            messageDiv.appendChild(content);
            chatBox.appendChild(messageDiv);
            chatBox.scrollTop = chatBox.scrollHeight;
            return content;
        }

        queryInput.addEventListener('keypress', (e) => {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                if (!sendButton.disabled) askQuestion();
            }
        });
    </script>
</body>
</html>
"""

# ================================
# INITIALIZE MODELS AND SERVICES
# ================================
try:
    logging.info("Initializing models and services...")

    embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
    vector_store = PineconeVectorStore.from_existing_index(
        index_name=PINECONE_INDEX_NAME,
        embedding=embed_model
    )

    logging.info("Loading local LLM (Google Gemma 7B)...")
    model_id = "meta-llama/Llama-2-13b-chat-hf"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True
    )

    # Load tokenizer & model
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",          # accelerate handles placement
        trust_remote_code=True
    )

    # ================================
    # Create text-generation pipeline
    # ================================
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.1
        # 🚫 No "device" arg here
    )

    # ================================
    # Example usage
    # ================================
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": "Summarize the advantages of using quantization in LLMs."}
    ]

    # Apply LLaMA chat template
    tokenized_chat = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        tokenized_chat,
        max_new_tokens=512,
        eos_token_id=tokenizer.eos_token_id
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    llm = HuggingFacePipeline(pipeline=pipe)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)

    MODELS_LOADED = True
    logging.info("✅ Initialization complete.")
except Exception as e:
    MODELS_LOADED = False
    llm = None
    vector_store = None
    text_splitter = None
    logging.error(f"🔥 Failed to initialize models: {e}")
    logging.error("The application will not be able to answer questions. Please check your API keys and runtime.")







In [ ]:
# ================================
# API ENDPOINTS (Updated for OCR and Session Handling)
# ================================
@app.get("/", response_class=HTMLResponse)
async def root():
    return HTMLResponse(content=html_content)

@app.post("/upload/")
async def upload(files: list[UploadFile] = File(...), session_id: str = Form(...)):
    if not MODELS_LOADED:
        raise HTTPException(status_code=503, detail="Model services are not available.")
    if not files:
        raise HTTPException(status_code=400, detail="No files were sent.")
    if not session_id:
        raise HTTPException(status_code=400, detail="A session ID is required.")

    logging.info(f"Starting upload for session ID: {session_id}")
    all_docs = []
    for file in files:
        file_content = await file.read()
        file_name = file.filename
        try:
            if file_name.lower().endswith(".pdf"):
                with fitz.open(stream=file_content, filetype="pdf") as doc:
                    for i, page in enumerate(doc):
                        text = page.get_text()
                        if text:
                            all_docs.append(Document(
                                page_content=text,
                                metadata={"file_name": file_name, "page_number": i + 1}
                            ))
            # ✅ NEW: OCR for images and screenshots
            # ✅ FIX: Robust OCR for images and screenshots
            elif file_name.lower().endswith((".png", ".jpg", ".jpeg")):
                image = Image.open(io.BytesIO(file_content))
                text = pytesseract.image_to_string(image)
                if text and text.strip():
                    all_docs.append(Document(
                        page_content=text,
                        metadata={"file_name": file_name, "type": "image_with_text"}
                    ))
                else:
                    # Fallback if no text is found
                    all_docs.append(Document(
                        page_content=f"This is an image named {file_name} that does not contain any readable text.",
                        metadata={"file_name": file_name, "type": "image"}
                    ))
        except Exception as e:
            logging.error(f"Failed to process file {file_name}: {e}")

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid content could be processed from the uploaded files.")

    split_docs = text_splitter.split_documents(all_docs)
    # ✅ NEW: Use the client-provided session ID as the namespace
    vector_store.add_documents(split_docs, namespace=session_id)
    logging.info(f"Successfully indexed {len(split_docs)} document chunks for session {session_id}.")
    return {"msg": f"Successfully indexed {len(split_docs)} document chunks."}

class QueryRequest(BaseModel):
    query: str
    session_id: str

@app.post("/query/")
async def query(req: QueryRequest):
    if not MODELS_LOADED:
        raise HTTPException(status_code=503, detail="Model services are not available.")

    user_query = req.query.strip().lower()
    session_id = req.session_id

    if not session_id:
        raise HTTPException(status_code=400, detail="A session ID is required for querying.")

    conversational_responses = {
        "hello": "Hi there! How can I help you with your documents today?",
        "hi": "Hello! I'm ready to answer your questions.",
        "how are you": "I'm just a set of algorithms, but I'm running perfectly! What can I do for you?",
        "thanks": "You're welcome! Let me know if you have any other questions.",
        "thank you": "You're welcome! Is there anything else you need help with?"
    }

    if user_query in conversational_responses:
        return {"answer": conversational_responses[user_query], "sources": []}

    # ✅ NEW: Use the client-provided session ID for retrieval
    retriever = vector_store.as_retriever(search_kwargs={"k": 3, "namespace": session_id})

    prompt_template = """
You are Spe, an expert assistant designed to assist users with analyzing and understanding content from uploaded files, including PDFs, documents, and images. Your responses must be clear, concise, and strictly based on the provided context, avoiding speculative information, metadata references (e.g., author names, DOIs), or repetitive boilerplate text. Do not include artificial section headers like "Summary:" or "Explanation:" unless explicitly requested.

Response Guidelines:
1. **Summarization Queries**: For queries like "summarize this document" or "what is this about," provide a concise summary in 4–6 sentences or 4–6 bullet points, capturing all key ideas without redundancy. Focus on the core content, intent, and main topics of the document or image. For images, summarize any extracted text or describe the visual content if no text is present.
2. **Text Extraction Queries** (e.g., "What text is written in this image/page?"): Return the extracted text accurately, cleaning up OCR noise (e.g., stray characters, misspellings) to ensure readability. Present the text in a clear, formatted manner, such as a list or paragraph.
3. **Title/Heading Queries**: Provide the exact title or heading if present in the context. If none exists, state: "No title or heading is present in the provided content."
4. **Content Description Queries** (e.g., "What is this document/pdf/image about?"): Provide a concise 1–2 sentence explanation of the main topic or purpose, distinguishing between PDFs and images if necessary.
5. **Specific Questions**: Answer directly and precisely using only the context. If the context lacks relevant information, respond with: "The provided content does not contain this information."
6. **Author Queries**: If author names are explicitly mentioned in the context, list them. Otherwise, respond with: "The provided content does not mention any author names."
7. **Image-Specific Queries**: For images, prioritize accurate text extraction using OCR results. If no text is extracted or the text is unreadable, describe the visual elements (e.g., "The image is a diagram of a bridge structure") and note the absence of readable text. If the query asks for specific image content (e.g., text, objects), verify the OCR output and correct any obvious errors before responding.

General Rules:
- Use natural, conversational language, avoiding phrases like "The text describes" unless necessary.
- Do not repeat information in multiple forms or add unnecessary disclaimers.
- Ensure responses are complete, non-repetitive, and directly address the query.
- For images, validate OCR output for accuracy and correct errors (e.g., misread characters) to avoid incorrect answers.
- Differentiate content types (PDF vs. image) only when relevant to the query.
- Maintain a professional yet approachable tone, ensuring clarity and accuracy.

Context:
{context}

Question: {question}

Answer:
"""
    prompt = PromptTemplate.from_template(prompt_template)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = qa_chain.invoke({"query": req.query})

    raw_answer = result["result"].strip()
    clean_answer = raw_answer.split("Answer:")[-1].strip()

    source_files = list(set([doc.metadata.get("file_name", "unknown") for doc in result["source_documents"]]))

    return {"answer": clean_answer, "sources": source_files}

In [ ]:
import uvicorn

public_url = ngrok.connect(8000)
print(f"✅ Your public URL is: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)